In [ ]:
import pandas as pd
import json
import warnings
import datetime
import os
from sqlalchemy import create_engine, inspect

In [ ]:
import os
print(os.getcwd())

In [ ]:
## Load configuration from json file
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

In [ ]:
schema = config["schema"]
host = config["host"]
user = config["user"]
password = config["password"]
port = config["port"]

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
connection_string

In [ ]:
# Create an engine
engine = create_engine(connection_string)
# Use an inspector to get a list of all table names
inspector = inspect(engine)
tables = inspector.get_table_names()

In [ ]:
# Import each table into a dictionary of DataFrames
dataframes = {}
for table in tables:
    dataframes[table] = pd.read_sql_table(table, engine)
    #here you can put the code to write the dataframes to either csv or excel files
    #directory = "/Users/user/data-science/general/review_ai_sdg/database/excel_tables/"
    dataframes[table].to_excel(f"{directory}{table}.xlsx", index=False)

### Alternative

In [ ]:
articles_df = pd.read_sql("articles", con=connection_string)

In [ ]:
articles_df.info()

In [ ]:
empiricalw_df = pd.read_sql("clusters", con=connection_string)
revconw_df = pd.read_sql("clusters_revcon", con=connection_string)
words_df = pd.concat([empiricalw_df, revconw_df], ignore_index=True)

In [ ]:
words_df.info()

In [ ]:
articles_df.info()

In [ ]:
merged_df = pd.merge(articles_df, words_df, on='eid', how='left')
merged_df.info()

In [ ]:
article_type_df = pd.read_sql("type_of_article", con=connection_string)
merged2_df = pd.merge(merged_df, article_type_df, on='eid', how='left')
merged2_df.info()

In [ ]:
df = merged2_df.drop(columns=["article_file"])
columna = df.pop("article_type")
df.insert(8, "article_type", columna)

In [ ]:
df['words'] = df['words'].fillna("excluded_from_word_analysis")
df['cluster_number'] = df['cluster_number'].fillna("excluded_from_word_analysis")
df['cluster_name'] = df['cluster_name'].fillna("excluded_from_word_analysis")

In [ ]:
df.info()

In [ ]:
df.to_excel("minimum_database.xlsx", index=False)

In [ ]:
revconw_df.to_excel("clusters_revcon.xlsx", index=False)